In [ ]:
import yt
import yt.extensions.geotiff

### Read GeoTIFF with yt

In [ ]:
filename = 'S2A_MSIL2A_20200601T113331_N0214_R080_T30UVG_20200601T123416_20m.tif'
ds = yt.load(filename, field_map="sentinel2_fields.yaml")

### Normalised Difference Vegetation Index (NDVI)

Here is an example of one thing we can easily calculate once the data is loaded into yt. The [Normalised Difference Vegetation Index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)! This is used to monitor vegetation density and health by comparing the reflectance of red (R) and near infrared (NIR) bands.
For our case here the NIR band is the 9 band in the GeoTIFF we created and the R band is 3.

In [ ]:
def _ndvi(field, data):
    return (data[('sentinel2', 'B8A')] - data[('sentinel2', 'B04')]) / \
           (data[('sentinel2', 'B8A')] + data[('sentinel2', 'B04')])

In [ ]:
ds.add_field("ndvi", function=_ndvi,
             units="", display_name='NDVI', take_log=False, sampling_type='cell')

In [ ]:
def _surface_area(field, data):
    return data['index', 'dx'] * data['index', 'dy']

In [ ]:
ds.add_field("surface_area", function=_surface_area,
             units="km**2", display_name="surface area", sampling_type='cell')

In [ ]:
p = yt.SlicePlot(ds, 'z', 'ndvi')
p.set_cmap(field="ndvi", cmap='RdYlGn')

In [ ]:
ad = ds.all_data()

In [ ]:
cr = ds.cut_region(ad, ["obj['sentinel2', 'B04'] >= 100", "obj['sentinel2', 'B8A'] >= 100"])

In [ ]:
profile = yt.ProfilePlot(cr, 'ndvi', 'surface_area', weight_field=None)
profile.show()

In [ ]:
cr['surface_area'][cr['ndvi'] >= 0.5].sum()

In [ ]:
ds.domain_width[:2].prod().to('km**2')

In [ ]:
phase = yt.PhasePlot(cr, ('sentinel2', 'B8A'), ('sentinel2', 'B04'), 'surface_area', weight_field=None)
phase.show()

Dan, this was me messing with the LAI from the SNAP biophysical processor. We don't have to include this unless you think of something cool to do with it.

In [ ]:
ds2 = yt.load('S2A_MSIL2A_20200601T113331_N0214_R080_T30UVG_20200601T123416_resampled_biophysical.tif')

In [ ]:
p2 = yt.SlicePlot(ds2, 'z', ('bands', '1'))
p2.set_log('1', False)
p2.set_zlim('1', 0, 4)